In [15]:
import pymongo
import pandas as pd
import re
import time
import datetime
from datetime import datetime, timedelta
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
threads = db.threads

In [6]:
df = pd.DataFrame.from_records(threads.find({'Label' :{'$ne': 0}}))

In [21]:
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [24]:
sentences = ie_preprocess(df['Body'][0])

In [27]:
sentence = sentences[0]

In [33]:
grammer = "NP: {<DT|PP\$>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammer)
result = cp.parse(sentence)

In [34]:
print(result)

(S
  For/IN
  those/DT
  who/WP
  aren/VBP
  t/NNS
  (NP aware/JJ new/JJ york/NN)
  is/VBZ
  going/VBG
  into/IN
  (NP a/DT housing/NN)
  crises/NNS
  to/TO
  rent/VB
  (NP a/DT space/NN)
  in/IN
  (NP the/DT shitty/JJ area/NN)
  of/IN
  queens/NNS
  is/VBZ
  around/IN
  15/CD
  000/CD
  (NP a/DT month/NN)
  for/IN
  commercial/JJ
  ./.)


In [ ]:
class ConsecutiveNPChunkTagger(nltk.TaggerI): [1]

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history) [2]
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train( [3]
            train_set, algorithm='megam', trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI): [4]
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags))